# with multi file reader

In [2]:
import glob
import re
import csv
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfTransformer as TF
from sklearn.naive_bayes import MultinomialNB as MNB
from sklearn.pipeline import Pipeline

In [3]:
def read_multiple_files(folder, file_ext):
    files = glob.glob('{}/*.{}'.format(folder, file_ext))
    data = []
    for file in files:
        with open(file, encoding='latin_1') as f:
            data.append(f.read())
    return data

In [4]:
file_exts = {'Ruby': 'jruby*', 'Clojure': 'clojure*', 'Scala': 'scala*', 'Python': 'py*', 
             'OCaml': 'ocaml*', 'Scheme': 'racket*', 'Common Lisp': 'sbcl*', 'Perl': 'perl*', 'PHP': 'hack*', 
             'C#': 'csharp*', 'Java': 'java', 'JavaScript': 'javascript*', 'C': 'gcc*', 'Haskell': 'ghc*'}
f_e_again = {'PHP': 'php*', 'Ruby': 'yarv*'}

In [5]:
X_language_train = []
y_language_train = []

X_language_test = []
y_language_test = []

In [6]:
for lang, ext in file_exts.items():
    get_data = read_multiple_files('bench',ext)
    X_language_train += get_data
    y_language_train += ([lang]*len(get_data))

    if lang == 'PHP':
        get_data = read_multiple_files('bench','php*')
        X_language_train += get_data
        y_language_train += ([lang]*len(get_data))     #can't use .append bc pipelines are fucking stupid

    if lang == 'Ruby':
        get_data = read_multiple_files('bench','yarv*')
        X_language_train += get_data
        y_language_train += ([lang]*len(get_data))


In [7]:
# X_language_train

In [8]:
# y_language_train

In [9]:
py_pipeline = Pipeline([("count", CV()), ("multi", MNB())])

In [10]:
# ("tfid", TF()),

In [11]:
py_pipeline.fit(X_language_train, y_language_train)


Pipeline(steps=[('count', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multi', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [12]:
py_pipeline.score(X_language_train,y_language_train)

0.99206349206349209

In [13]:
for lang, ext in file_exts.items():
    get_data = read_multiple_files('bench2',ext)
    X_language_test += get_data
    y_language_test += ([lang]*len(get_data))

    if lang == 'PHP':
        get_data = read_multiple_files('bench2','php*')
        X_language_test += get_data
        y_language_test += ([lang]*len(get_data))     #can't use .append bc pipelines are fucking stupid

    if lang == 'Ruby':
        get_data = read_multiple_files('bench2','yarv*')
        X_language_test += get_data
        y_language_test += ([lang]*len(get_data))


In [14]:
def test_one_program(file):
    data = []
    with open(file, encoding='latin_1') as f:
        reader = csv.reader(f)
        for row in reader:
            data.append(f.read())
    return data

In [15]:
X_language_test = test_one_program('bench2/mandelbrot.clojure-5.clojure')
# X_language_test

In [16]:
py_pipeline.predict(X_language_test)   # this is a real Clojure program

array(['Clojure'], 
      dtype='<U11')

In [17]:
X_single_language_test = ''

for lang, ext in file_exts.items():
    get_data = read_multiple_files('test_one_each',ext)
    X_single_language_test = get_data

    if lang == 'PHP':
        print(lang,py_pipeline.predict(X_single_language_test))
        get_data = read_multiple_files('test_one_each','php*')
        X_single_language_test = get_data

    if lang == 'Ruby':
        print(lang,py_pipeline.predict(X_single_language_test))
        get_data = read_multiple_files('test_one_each','yarv*')
        X_single_language_test = get_data

    print(lang,py_pipeline.predict(X_single_language_test))

Scheme ['Scheme']
OCaml ['OCaml']
Clojure ['Clojure']
Perl ['Perl']
C# ['C#']
Python ['Clojure']
C ['C']
Scala ['Scala']
Haskell ['Haskell']
Ruby ['Ruby']
Ruby ['Ruby']
PHP ['PHP']
PHP ['PHP']
Common Lisp ['Common Lisp']
JavaScript ['JavaScript']
Java ['Java']
